In [66]:
using LinearAlgebra

function csminwel(fcn, x0, H0, grad, crit, nit; kwargs...) 
    #= 
    This function accepts inputs the following inputs:
    fcn: Julia function describing the function to be minimized
    x0: Initial point
    H0: Initial Hessian
    grad: Gradient function (if available, otherwise pass "nothing")
    crit: Objective improvement threshold condition
    nit: Max. number of iterations before terminating
    =#
    
    
    nx = length(x0)
    x = copy(x0)
    H = Matrix(H0)
    NumGrad = isnothing(grad)
    f = fcn(x; kwargs...)

    if f > 1e50
        error("Bad initial parameter.")
    end

    g, badg = NumGrad ? numerical_gradient(fcn, x; kwargs...) : grad(x; kwargs...)

    itct = 0
    fcount = 0
    done = false
    
    # --- FIX START: Initialize retcode1 here ---
    retcode1 = 0 
    # --- FIX END ---

    while !done
        println("-----------------")
        println("Iteration: $itct")
        println("Function value: $f")
        println("x: $x")

        itct += 1

        # Because retcode1 was initialized above, this update affects the outer variable
        f1, x1, fc, retcode1 = csminit(fcn, x, f, g, badg, H; kwargs...)
        fcount += fc
        print("Return Code: $retcode1\nEval Count: $fcount\n")
        
        g1, badg1 = NumGrad ? numerical_gradient(fcn, x1; kwargs...) : grad(x1; kwargs...)

        if retcode1 != 1
            if retcode1 in [2, 4]
                wall1 = true
            else
                wall1 = badg1
            end

            if wall1 && nx > 1
                println("Cliff detected. Perturbing search direction.")
                H_cliff = H + Diagonal(rand(nx) .* diag(H))
                f2, x2, fc, retcode2 = csminit(fcn, x, f, g, badg, H_cliff; kwargs...)
                fcount += fc

                if f2 < f
                    g2, badg2 = NumGrad ? numerical_gradient(fcn, x2; kwargs...) : grad(x2; kwargs...)
                    wall2 = badg2
                    if wall2
                        println("Cliff again. Traversing.")
                        if norm(x2 - x1) < 1e-13
                            f3, x3, badg3, retcode3 = f, x, true, 101
                        else
                            g_cliff = ((f2 - f1) / (norm(x2 - x1)^2)) * (x2 - x1)
                            f3, x3, fc, retcode3 = csminit(fcn, x, f, g_cliff, false, Matrix(I(nx)); kwargs...)
                            fcount += fc
                            g3, badg3 = NumGrad ? numerical_gradient(fcn, x3; kwargs...) : grad(x3; kwargs...)
                        end
                    else
                         f3, x3, badg3, retcode3 = f, x, true, 101
                    end
                else
                    f3, x3, badg3, retcode3 = f, x, true, 101
                end
            else
                 f2, f3, badg2, badg3, retcode2, retcode3 = f, f, true, true, 101, 101
            end
        else
            f2, f3, f1, retcode2, retcode3 = f, f, f, retcode1, retcode1
        end

        if !badg && !badg1 && abs(f1 - f) >= crit
            H = bfgsi(H, g1 - g, x1 - x)
        end

        println("Improvement: $(f - f1)")

        if itct >= nit || abs(f - f1) < crit
            done = true
        end
        
        # Update state for next iteration
        f, x, g = f1, x1, g1
    end

    return f, x, g, H, itct, fcount, retcode1
end

# ------------------------------------ HELPER FUNCTIONS --------------------------------------
function numerical_gradient(fcn, x; kwargs...)
    eps = 1e-8
    g = zeros(length(x))
    for i in 1:length(x)
        x_forward = copy(x)
        x_backward = copy(x)
        x_forward[i] += eps
        x_backward[i] -= eps
        g[i] = (fcn(x_forward; kwargs...) - fcn(x_backward; kwargs...)) / (2 * eps)
    end
    return g, false
end

function csminit(fcn, x0, f0, g0, badg, H0; kwargs...)
    ANGLE = 0.005
    THETA = 0.3
    FCHANGE = 1000
    MINLAMB = 1e-9
    MINDFAC = 0.01
    fcount = 0
    lambda = 1.0
    xhat = x0
    fhat = f0
    g = g0
    gnorm = norm(g)

    if gnorm < 1e-12 && !badg
        return fhat, xhat, fcount, 1
    end

    dx = -H0 * g
    dxnorm = norm(dx)
    if dxnorm > 1e12
        dx *= FCHANGE / dxnorm
    end

    dfhat = dot(dx, g0)
    if !badg
        a = -dfhat / (gnorm * dxnorm)
        if a < ANGLE
            dx -= (ANGLE * dxnorm / gnorm + dfhat / (gnorm^2)) * g
            dx *= dxnorm / norm(dx)
            dfhat = dot(dx, g)
        end
    end

    done = false
    factor = 3.0
    shrink = true
    lambda_min = 0.0
    lambda_max = Inf
    lambda_peak = 0.0
    f_peak = f0
    lambdahat = 0.0

    while !done
        dxtest = x0 + lambda * dx
        f = fcn(dxtest; kwargs...)
        fcount += 1
        if f < fhat
            fhat = f
            xhat = dxtest
            lambdahat = lambda
        end
        shrink_signal = (!badg && (f0 - f < max(-THETA * dfhat * lambda, 0))) || (badg && (f0 - f) < 0)
        grow_signal = !badg && (lambda > 0 && (f0 - f > -(1 - THETA) * dfhat * lambda))

        if shrink_signal && (lambda > lambda_peak || lambda < 0)
            if lambda > 0 && (!shrink || lambda / factor <= lambda_peak)
                shrink = true
                factor = factor^0.6
                while lambda / factor <= lambda_peak
                    factor = factor^0.6
                end
                if abs(factor - 1) < MINDFAC
                    return fhat, xhat, fcount, if abs(lambda) < 4.0 2 else 7 end
                end
            end
            lambda = lambda / factor
            if abs(lambda) < MINLAMB
                if lambda > 0 && f0 <= fhat
                    lambda = -lambda * factor^6
                else
                    return fhat, xhat, fcount, if lambda < 0 6 else 3 end
                end
            end
        elseif grow_signal && lambda > 0 || (shrink_signal && (lambda <= lambda_peak && lambda > 0))
            if shrink
                shrink = false
                factor = factor^0.6
                if abs(factor - 1) < MINDFAC
                    return fhat, xhat, fcount, if abs(lambda) < 4.0 4 else 7 end
                end
            end
            if f < f_peak && lambda > 0
                f_peak = f
                lambda_peak = lambda
                if lambda_max <= lambda_peak
                    lambda_max = lambda_peak * factor^2
                end
            end
            lambda = lambda * factor
            if abs(lambda) > 1e20
                return fhat, xhat, fcount, 5
            end
        else
            done = true
            return fhat, xhat, fcount, if factor < 1.2 7 else 0 end
        end
    end
    return fhat, xhat, fcount, 0
end

function bfgsi(H, delta_g, delta_x) #-------------BFGS algorithm for hessian calculation------------
    rho = 1.0 / (dot(delta_g, delta_x))
    # Safety check for division by zero in BFGS update
    if isinf(rho) || isnan(rho)
        return H
    end
    V = I(length(delta_x)) - rho * (delta_x * delta_g')
    return V * H * V' + rho * (delta_x * delta_x')
end



function sims_optimize(fcn, x0, H0, grad, crit=1e-9, nit=100; kwargs...) 
    println("STARTING OPTIMIZATION...")
    results=csminwel(fcn, x0, H0, grad, crit, nit)
    println("\nFINAL RESULTS:")
    println("Min Value: ", results[1])
    println("Min X: ", results[2])
    return results[1],results[2]
end

sims_optimize (generic function with 3 methods)

In [68]:
#Testing Performance on common examples
rosenbrock=x->(12.0-x[1])^2+0.5*((x[2]-x[1]^2)^2) #Rosenbruck function (easy for the algorithm to handle)
x0=[-100.8,0.1];
H0=I(2);
fval=rosenbrock(x0);

sims_optimize(rosenbrock, x0, H0, nothing)



STARTING OPTIMIZATION...
-----------------
Iteration: 0
Function value: 5.16310103858e7
x: [-100.8, 0.1]
Return Code: 0
Eval Count: 11
Improvement: 5.163088295946416e7
-----------------
Iteration: 1
Function value: 127.426335832148
x: [3.2790995610097724, 0.6161996016880061]
Return Code: 0
Eval Count: 12
Improvement: 50.252969046240025
-----------------
Iteration: 2
Function value: 77.17336678590797
x: [3.225187135459662, 10.995171722342741]
Return Code: 0
Eval Count: 13
Improvement: 0.24683342533235475
-----------------
Iteration: 3
Function value: 76.92653336057562
x: [3.229441302852498, 10.341729716224595]
Return Code: 0
Eval Count: 19
Improvement: 0.24310861391994365
-----------------
Iteration: 4
Function value: 76.68342474665567
x: [3.256419490797263, 9.921296999133167]
Return Code: 0
Eval Count: 25
Improvement: 5.513527863525923
-----------------
Iteration: 5
Function value: 71.16989688312975
x: [4.126773467438723, 12.744888590784187]
Return Code: 0
Eval Count: 28
Improvement: 1

(5.51293986216348e-12, [11.999999816461864, 143.9999989054484])

In [43]:
g=x->exp(abs(x[3]-.85)*(x[1]-.2)^2+x[1]*exp(x[2]*x[3]-.1))

x0=[-0.0,0.0,0.0];
H0=I(3);
fval=g(x0);

sims_optimize(g, x0, H0, nothing, 1e-9, 100)




STARTING OPTIMIZATION...
-----------------
Iteration: 0
Function value: 1.034584606728118
x: [-0.0, 0.0, 0.0]
Return Code: 0
Eval Count: 2
Improvement: 0.0794692677641139
-----------------
Iteration: 1
Function value: 0.955115338964004
x: [-0.19479069838605292, 0.0, 0.013794461869072922]
Return Code: 7
Eval Count: 11
Improvement: 0.50845580293049
-----------------
Iteration: 2
Function value: 0.4466595360335141
x: [-0.9779186892293125, 0.014853134003599958, 0.9155964499776079]
Return Code: 0
Eval Count: 13
Improvement: 0.044571621883696455
-----------------
Iteration: 3
Function value: 0.40208791414981765
x: [-0.9799558263778712, 0.08003193442343741, 0.8240937840939946]
Return Code: 0
Eval Count: 14
Improvement: 0.40182007877984727
-----------------
Iteration: 4
Function value: 0.00026783536997035607
x: [-2.405105347951616, 1.5324146160020762, 0.8875760531619947]
Return Code: 0
Eval Count: 19
Improvement: 0.00017533632519917797
-----------------
Iteration: 5
Function value: 9.249904477

(-1.249999129005749, [-1.000000009132223, -0.49906917421608227])

In [62]:
wheeler(x, a=1.5) = -exp(-(x[1]*x[2] - a)^2 -(x[2]-a)^2) 

sims_optimize(wheeler, [-1.0,3.0], [1.0 1.0;1.0 1.0], nothing, 1e-9, 100)

STARTING OPTIMIZATION...
-----------------
Iteration: 0
Function value: -1.6918979226151304e-10
x: [-1.0, 3.0]
Return Code: 0
Eval Count: 32
Improvement: 0.00018020586374944343
-----------------
Iteration: 1
Function value: -0.00018020603293923568
x: [-0.016786095934483325, 3.983213904065517]
Return Code: 0
Eval Count: 37
Improvement: 1.265068189413123e-7
-----------------
Iteration: 2
Function value: -0.000180332539758177
x: [-0.023103575217310223, 3.9672522569953155]
Return Code: 0
Eval Count: 42
Improvement: 1.2912280061269437e-7
-----------------
Iteration: 3
Function value: -0.00018046166255878968
x: [-0.016776098334139386, 3.982956320666975]
Return Code: 0
Eval Count: 47
Improvement: 1.2512572095986366e-7
-----------------
Iteration: 4
Function value: -0.00018058678827974955
x: [-0.010995292992892352, 3.9971022409180046]
Return Code: 0
Eval Count: 52
Improvement: 1.2011518696241805e-7
-----------------
Iteration: 5
Function value: -0.00018070690346671197
x: [-0.005149840677839095

Improvement: 1.850734351009761e-8
-----------------
Iteration: 57
Function value: -0.0001829680106425577
x: [0.17647173026937077, 4.340309484478734]
Return Code: 0
Eval Count: 380
Improvement: 1.816211782294905e-8
-----------------
Iteration: 58
Function value: -0.00018298617276038065
x: [0.1787083406644892, 4.3429011747563875]
Return Code: 0
Eval Count: 388
Improvement: 1.7815440763873118e-8
-----------------
Iteration: 59
Function value: -0.00018300398820114453
x: [0.18096361267180036, 4.345477012616719]
Return Code: 0
Eval Count: 396
Improvement: 1.7467413696204857e-8
-----------------
Iteration: 60
Function value: -0.00018302145561484073
x: [0.18323789833649534, 4.348036433051474]
Return Code: 0
Eval Count: 404
Improvement: 1.711880410199657e-8
-----------------
Iteration: 61
Function value: -0.00018303857441894273
x: [0.1855315413251129, 4.350578830622929]
Return Code: 0
Eval Count: 412
Improvement: 1.6769612562407728e-8
-----------------
Iteration: 62
Function value: -0.000183055

(-0.00018356405367018027, [0.24193803353979604, 4.400592219339656])

In [56]:
wheeler([-1.0,-0.5])

-0.006737946999085467